In [ ]:
!rm -rf ./data/trainingData0720

# Environment

In [ ]:
import tensorflow as tf
import pathlib
import random
import IPython.display as display
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import gc
from keras.optimizers import *
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from functools import partial
import cv2

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
print(tf.__version__)

# Set Parameter

In [ ]:
img_rows = 96
img_cols = 96
n_class = 2

# Download data from google drive

In [ ]:
%%time
zip_path = '/content/drive/MyDrive/ARC _AIoT 共用雲端/datasetclearplushand0721.zip'
!cp "{zip_path}" .
!unzip -q datasetclearplushand0721.zip -d './data'
!rm datasetclearplushand0721.zip

# List all directory

In [ ]:
data_root = pathlib.Path('./data')

In [ ]:
for item in data_root.iterdir():
  print(item)

In [ ]:
all_image_paths = list(data_root.glob('*/*')) 
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

# Prepare Dataset

In [ ]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

In [ ]:
label_to_index = dict((name, index) for index, name in enumerate(label_names))
label_to_index

In [ ]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

test_labels = to_categorical(all_image_labels, n_class, dtype = 'float32')
print("First 10 labels indices: ", all_image_labels[:20])
# print(test_labels[:20])

In [ ]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=1)
  image = tf.image.resize(image, [img_rows, img_cols])

  return image

In [ ]:
def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, test_labels))
# 元组被解压缩到映射函数的位置参数中
def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

image_label = ds.map(load_and_preprocess_from_path_label)
print(image_label)

In [ ]:
BATCH_SIZE = 32


setDs = image_label.shuffle(buffer_size=image_count)
setDs = setDs.repeat()
setDs_Batch = setDs.batch(BATCH_SIZE)

setDs = setDs.prefetch(buffer_size=AUTOTUNE)
print(setDs)
print(setDs_Batch)

## Separate training and testing set

In [ ]:
trainingSet = setDs.skip(int(image_count*0.2))
trainingSet_Batch = setDs_Batch.take(int(image_count*0.2))

testingSet = setDs.take(int(image_count*0.2))
testingSet_Batch = setDs_Batch.take(int(image_count*0.2))

# Data Visualization

In [ ]:
def plot_dataset(dataset, num=5):
    plt.figure(figsize=(15, 15))
    plot_index = 0
    for image, label in dataset.take(num):
        image = image.numpy()
        label = label.numpy()
        
        plot_index+=1
        plt.subplot(3, 5, plot_index)
        plt.imshow(image.squeeze(),'gray')

plot_dataset(trainingSet, 15)
gc.collect()

# Image Augment

In [ ]:
pip install tensorflow_addons

In [ ]:
import tensorflow_addons as tfa

In [ ]:
def image_transpose(image):
    rand = tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32) 
    image = tf.cond(rand < 0.5, 
                    lambda: tf.identity(image), 
                    lambda: tf.image.transpose(image)) 
    return image

def image_flip(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image

def image_rotate(image):
    # image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    rand = tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32) 
    def random_rotate(image):
        image = tfa.image.rotate(
            image, tf.random.uniform(shape=[], minval=-30 * np.pi / 180, maxval=30 * np.pi / 180, dtype=tf.float32))
        return image
    
    image = tf.cond(rand < 0.5, 
                    # lambda: tf.identity(image), 
                    lambda: random_rotate(image),
                    lambda: random_rotate(image)) 
    return image  

def image_color(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.random_saturation(image, lower=0.5, upper=3)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1)
    image = tf.image.random_hue(image, max_delta=0.03)
    image = tf.clip_by_value(image, clip_value_min=0, clip_value_max=1)
    return image

def image_inversion(image: tf.Tensor) -> tf.Tensor:
    rand = tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32)
    image = tf.cond(rand < 0.8, 
                    lambda: tf.identity(image), 
                    lambda: tf.math.add(tf.math.multiply(image, -1), 1))
    return image

def image_zoom(image: tf.Tensor, min_zoom=0.8, max_zoom=1.0) -> tf.Tensor:
    image_width, image_height, image_colors = image.shape
    crop_size = (image_width, image_height)

    # Generate crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(min_zoom, max_zoom, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize(
            [img],
            boxes=boxes,
            box_indices=np.zeros(len(scales)),
            crop_size=crop_size
        )
        # Return a random crop
        return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]

    choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: image, lambda: random_crop(image))

In [ ]:
def augment_data(image, label):
    # image = image_flip(image)
    # image = image_color(image)
    # image = image_zoom(image)
    # image = image_transpose(image)
    image = image_inversion(image)
    # image = image_rotate(image)
    return image, label

## Augment Visualization

In [ ]:
trainingSet = trainingSet.map(augment_data)

plot_dataset(trainingSet,15)

# Create Model

## **INCEPTION_NET**

In [ ]:
conv1x1 = partial(layers.Conv2D, kernel_size=1, activation='relu')
conv3x3 = partial(layers.Conv2D, kernel_size=3, padding='same', activation='relu')
conv5x5 = partial(layers.Conv2D, kernel_size=5, padding='same', activation='relu')

def inception_module(in_tensor, c1, c3_1, c3, c5_1, c5, pp):
  conv1 = conv1x1(c1)(in_tensor)
  conv3_1 = conv1x1(c3_1)(in_tensor)
  conv3 = conv3x3(c3)(conv3_1) 
  conv5_1 = conv1x1(c5_1)(in_tensor) 
  conv5 = conv5x5(c5)(conv5_1) 
  pool_conv = conv1x1(pp)(in_tensor) 
  pool = layers.MaxPool2D(3, strides=1, padding='same')(pool_conv) 
  merged = layers.Concatenate(axis=-1)([conv1, conv3, conv5, pool]) 
  return merged 

def aux_clf(in_tensor): 
  avg_pool = layers.AvgPool2D(5, 3)(in_tensor)
  conv = conv1x1(128)(avg_pool)
  flattened = layers.Flatten()(conv)
  dense = layers.Dense(1024, activation='relu')(flattened)
  dropout = layers.Dropout(0.7)(dense)
  out = layers.Dense(n_class, activation='softmax')(dropout)
  return out

def inception_net(in_shape=(224,224,3), n_classes=1000, opt='sgd'): 
  in_layer = layers.Input(in_shape) 
  rescale = layers.experimental.preprocessing.Rescaling(1./255),
  conv1 = layers.Conv2D(64, 7, strides=2, activation='relu', padding='same')(in_layer) 
  pad1 = layers.ZeroPadding2D()(conv1) 
  pool1 = layers.MaxPool2D(3, 2)(pad1) 
  conv2_1 = conv1x1(64)(pool1) 
  conv2_2 = conv3x3(192)(conv2_1) 
  pad2 = layers.ZeroPadding2D()(conv2_2) 
  pool2 = layers.MaxPool2D(3, 2)(pad2) 
  inception3a = inception_module(pool2, 64, 96, 128, 16, 32, 32) 
  inception3b = inception_module(inception3a, 128, 128, 192, 32, 96, 64) 
  pad3 = layers.ZeroPadding2D()(inception3b) 
  pool3 = layers.MaxPool2D(3, 2)(pad3) 
  inception4a = inception_module(pool3, 192, 96, 208, 16, 48, 64) 
  inception4b = inception_module(inception4a, 160, 112, 224, 24, 64, 64) 
  inception4c = inception_module(inception4b, 128, 128, 256, 24, 64, 64) 
  inception4d = inception_module(inception4c, 112, 144, 288, 32, 48, 64) 
  inception4e = inception_module(inception4d, 256, 160, 320, 32, 128, 128) 
  pad4 = layers.ZeroPadding2D()(inception4e) 
  pool4 = layers.MaxPool2D(3, 2)(pad4) 
  aux_clf1 = aux_clf(inception4a) 
  aux_clf2 = aux_clf(inception4d) 
  inception5a = inception_module(pool4, 256, 160, 320, 32, 128, 128) 
  inception5b = inception_module(inception5a, 384, 192, 384, 48, 128, 128) 
  pad5 = layers.ZeroPadding2D()(inception5b) 
  pool5 = layers.MaxPool2D(3, 2)(pad5) 
  avg_pool = layers.GlobalAvgPool2D()(pool5) 
  dropout = layers.Dropout(0.4)(avg_pool) 
  preds = layers.Dense(n_class, activation='softmax')(dropout) 
  model = Model(in_layer, [preds, aux_clf1, aux_clf2]) 
  # model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"]) 
  return model 

In [ ]:
model = inception_net(in_shape=(img_rows, img_cols, 1), n_classes=n_class) 

## Resnet50

In [ ]:

backend = None
models = None
keras_utils = None


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    # if backend.image_data_format() == 'channels_last':
    #     bn_axis = 3
    # else:
    #     bn_axis = 1
    bn_axis = 1
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size,
                      padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        strides: Strides for the first conv layer in the block.
    # Returns
        Output tensor for the block.
    Note that from stage 3,
    the first conv layer at main path is with strides=(2, 2)
    And the shortcut should have strides=(2, 2) as well
    """
    filters1, filters2, filters3 = filters
    # if backend.image_data_format() == 'channels_last':
    #     bn_axis = 3
    # else:
    #     bn_axis = 1

    bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1), strides=strides,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = layers.Conv2D(filters3, (1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x


def ResNet50(include_top=True,
             weights='imagenet',
             input_tensor=None,
             input_shape=None,
             pooling=None,
             classes=10,
             **kwargs):
    """Instantiates the ResNet50 architecture.
    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 32.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional block.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional block, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    global backend, layers, models, keras_utils
    # backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    bn_axis = 1

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    # if backend.image_data_format() == 'channels_last':
    #     bn_axis = 3
    # else:
    #     bn_axis = 1

    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    if include_top:
        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        x = layers.Dense(classes, activation='softmax', name='fc1000')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D()(x)
        else:
            warnings.warn('The output shape of `ResNet50(include_top=False)` '
                          'has been changed since Keras 2.2.0.')

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = keras_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    return model

In [ ]:
model = ResNet50(include_top=True,
             weights='imagenet',
             input_tensor=None,
             input_shape=(img_rows, img_cols, 1),
             pooling=None,
             classes=n_class)

## **GRAY RESNET50**

In [ ]:
import numpy as np
import warnings
import os
import tensorflow as tf

from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.applications.imagenet_utils import preprocess_input
from keras.engine.topology import get_source_inputs
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

def identity_block(input_tensor, kernel_size, filters, stage, block):
  
    filters1, filters2, filters3 = filters
    bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
   
    filters1, filters2, filters3 = filters
    bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def grayResNet50(include_top=True, weights=None,
             input_tensor=None, input_shape=None,
             pooling=None,
             classes=2):
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    #if weights == 'imagenet' and include_top and classes != 15:
    #    raise ValueError('If using `weights` as imagenet with `include_top`'
    #                     ' as true, `classes` should be 15')

    # Determine proper input shape
    img_input = Input(shape=input_shape)
    
    bn_axis = 1

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D((3, 3), name='avg_pool')(x)

    if include_top:
        x = Flatten()(x)
        x = Dense(classes, activation='softmax', name='fc2')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    return model

In [ ]:
model = grayResNet50(include_top=True,
            input_shape=(img_rows, img_cols, 1),
            classes=n_class)

## VGG

In [ ]:
def VGG16Net(width, height, depth, classes):
    
    model = Sequential()
    
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=(width, height, depth),padding='same',activation='relu'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(3,2),strides=(1,1),padding='same',activation='relu'))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17,activation='softmax'))
    
    return model

In [ ]:
model = VGG16Net(img_rows,img_cols,1,n_class)

## EDGE　IMPULSE

In [ ]:
def EDGE(width, height, depth, classes):
  model = Sequential()
  model.add(layers.Conv2D(32, kernel_size=3, activation='relu', kernel_constraint=tf.keras.constraints.MaxNorm(1), padding='same', input_shape=(width, height, 1)))
  model.add(layers.MaxPooling2D(pool_size=2, strides=2, padding='same'))
  model.add(layers.Conv2D(16, kernel_size=3, activation='relu', kernel_constraint=tf.keras.constraints.MaxNorm(1), padding='same'))
  model.add(layers.MaxPooling2D(pool_size=2, strides=2, padding='same'))
  model.add(layers.Flatten())
  model.add(layers.Dropout(0.25))
  model.add(layers.Dense(n_class, activation='softmax', name='y_pred'))

  return model

In [ ]:
model = EDGE(img_rows,img_cols,1,n_class)

## MNIST

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense
from tensorflow.keras.layers import Activation, BatchNormalization, Flatten
from tensorflow.keras.models import Sequential
# Model create #1
def mnist(width, height, depth, classes):
  input_shape = (width, height, depth)
  model=Sequential()
  #Conv1
  model.add(Conv2D(filters=16, 
            kernel_size=(3, 3), 
            padding="same",  
            input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(MaxPooling2D())

  #Conv2
  model.add(Conv2D(filters=32, 
            kernel_size=(3, 3), 
            padding="same", 
            input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(MaxPooling2D())

  #Conv3
  model.add(Conv2D(filters=32, 
            kernel_size=(3, 3), 
            padding="same", 
            input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(MaxPooling2D())

  #FC1
  model.add(Flatten())
  model.add(Dense(64))
  model.add(BatchNormalization())
  model.add(Activation("relu"))

  #FC2
  model.add(Dense(classes))
  model.add(Activation("softmax"))

  return model

In [ ]:
model = mnist(img_rows,img_cols,1,n_class)

## EFFNET

In [ ]:
from keras.models import Model
from keras.layers import *
from keras.activations import *
from keras.callbacks import *


def get_post(x_in):
    # x = LeakyReLU()(x_in)
    x = BatchNormalization()(x_in)
    return x

def get_block(x_in, ch_in, ch_out):
    x = Conv2D(ch_in,
               kernel_size=(1, 1),
               padding='same',
               use_bias=False)(x_in)
    x = get_post(x)

    x = DepthwiseConv2D(kernel_size=(1, 3), padding='same', use_bias=False)(x)
    x = get_post(x)
    x = MaxPool2D(pool_size=(2, 1),
                  strides=(2, 1))(x) # Separable pooling

    x = DepthwiseConv2D(kernel_size=(3, 1),
                        padding='same',
                        use_bias=False)(x)
    x = get_post(x)

    x = Conv2D(ch_out,
               kernel_size=(2, 1),
               strides=(1, 2),
               padding='same',
               use_bias=False)(x)
    x = get_post(x)

    return x


def Effnet(input_shape, nb_classes, include_top=True, weights=None):
    x_in = Input(shape=input_shape)

    x = get_block(x_in, 32, 64)
    x = get_block(x, 64, 128)
    x = get_block(x, 128, 256)

    if include_top:
        x = Flatten()(x)
        x = Dense(nb_classes, activation='softmax')(x)

    model = Model(inputs=x_in, outputs=x)

    if weights is not None:
        model.load_weights(weights, by_name=True)

    return model

In [ ]:
model = Effnet((img_rows,img_cols,1),n_class)

## Mobil

In [ ]:
from keras.optimizers import *
from keras.models import Model
from keras.layers import *
from keras.activations import *
from keras.callbacks import *


def get_conv_block(tensor, channels, strides, alpha=1.0, name=''):
    channels = int(channels * alpha)

    x = Conv2D(channels,
               kernel_size=(3, 3),
               strides=strides,
               use_bias=False,
               padding='same',
               name='{}_conv'.format(name))(tensor)
    x = BatchNormalization(name='{}_bn'.format(name))(x)
    x = Activation('relu', name='{}_act'.format(name))(x)
    return x


def get_dw_sep_block(tensor, channels, strides, alpha=1.0, name=''):
    """Depthwise separable conv: A Depthwise conv followed by a Pointwise conv."""
    channels = int(channels * alpha)

    # Depthwise
    x = DepthwiseConv2D(kernel_size=(3, 3),
                        strides=strides,
                        use_bias=False,
                        padding='same',
                        name='{}_dw'.format(name))(tensor)
    x = BatchNormalization(name='{}_bn1'.format(name))(x)
    x = Activation('relu', name='{}_act1'.format(name))(x)

    # Pointwise
    x = Conv2D(channels,
               kernel_size=(1, 1),
               strides=(1, 1),
               use_bias=False,
               padding='same',
               name='{}_pw'.format(name))(x)
    x = BatchNormalization(name='{}_bn2'.format(name))(x)
    x = Activation('relu', name='{}_act2'.format(name))(x)
    return x


def MobileNet(shape, num_classes, alpha=1.0, include_top=True, weights=None):
    x_in = Input(shape=shape)

    x = get_conv_block(x_in, 32, (2, 2), alpha=alpha, name='initial')

    layers = [
        (64, (1, 1)),
        (128, (2, 2)),
        (128, (1, 1)),
        (256, (2, 2)),
        (256, (1, 1)),]
    #     (512, (2, 2)),
    #     *[(512, (1, 1)) for _ in range(5)],
    #     (1024, (2, 2)),
    #     (1024, (2, 2))
    # ]

    for i, (channels, strides) in enumerate(layers):
        x = get_dw_sep_block(x, channels, strides, alpha=alpha, name='block{}'.format(i))

    if include_top:
        x = GlobalAvgPool2D(name='global_avg')(x)
        x = Dense(num_classes, activation='softmax', name='softmax')(x)

    model = Model(inputs=x_in, outputs=x)

    if weights is not None:
        model.load_weights(weights, by_name=True)

    return model

In [ ]:
model = MobileNet((img_rows,img_cols,1),n_class)

# **Model Summary**

In [ ]:
print(model.summary())

# Train model

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

steps_per_epoch = image_count // BATCH_SIZE
validation_steps = image_count*0.2 // BATCH_SIZE

print('steps_per_epoch:', steps_per_epoch)
print('validation_steps:', validation_steps)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy')

In [ ]:
epoch = 20

In [ ]:
history = model.fit(trainingSet_Batch,
           validation_data=testingSet_Batch,
           epochs=epoch, 
           validation_steps=validation_steps,
           verbose=2)
          #  callbacks=[early_stopping],
          #  steps_per_epoch=steps_per_epoch,

# Plot history

In [ ]:
print(history.history.keys())

In [ ]:
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

# Predict image

In [ ]:
!pip install opencv-python
from google.colab.patches import cv2_imshow

In [ ]:
predictimg = random.choice(all_image_paths)
print(predictimg)
img = cv2.imread('./'+predictimg,cv2.IMREAD_UNCHANGED)
# gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

cv2_imshow(img)
gray = cv2.resize(img, (img_rows, img_cols), interpolation=cv2.INTER_CUBIC)
reshapeimg = gray.reshape(1,img_rows, img_cols, 1) 
transimg = reshapeimg.astype('float32')
predict = model.predict(transimg)
print(predict)
print(np.argmax(predict))

In [ ]:
# Save weights of this model
model.save_weights('./my_modelmobilmixset.h5')

# Draw confusion matrix

## Prepare Evaluation Data

In [ ]:
%%time
zip_path = '/content/drive/MyDrive/ARC _AIoT 共用雲端/datasetmix0722.zip'
!cp "{zip_path}" .
!unzip -q datasetmix0722.zip -d './testingdata'
!rm datasetmix0722.zip

In [ ]:
testingdata_root = pathlib.Path('./testingdata')

In [ ]:
for item in testingdata_root.iterdir():
  print(item)

In [ ]:
test_image_paths = list(testingdata_root.glob('*/*')) 
test_image_paths = [str(path) for path in test_image_paths]

In [ ]:
testinglabel_names = sorted(item.name for item in testingdata_root.glob('*/') if item.is_dir())

In [ ]:
testinglabel_to_index = dict((name, index) for index, name in enumerate(testinglabel_names))
testinglabel_to_index

In [ ]:
test_image_labels = [testinglabel_to_index[pathlib.Path(path).parent.name] for path in test_image_paths]

testing_labels = to_categorical(test_image_labels, n_class, dtype = 'float32')
print("First 10 labels indices: ", test_image_labels[:20])

In [ ]:
from sklearn.metrics import classification_report

import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import cv2

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

randomlist = random.sample(range(0, image_count), int(image_count)) 
y_predict = []
y_true = test_image_labels
for i,j in enumerate(test_image_paths):
  img = cv2.imread('./'+j,cv2.IMREAD_UNCHANGED)
  gray = cv2.resize(img, (img_rows, img_cols), interpolation=cv2.INTER_CUBIC)
  reshapeimg = gray.reshape(1,img_rows, img_cols, 1) 
  transimg = reshapeimg.astype('float32')
  y_predict.append(np.argmax(model.predict(transimg)))
target_names = ['0', '1']

In [ ]:
print(classification_report(y_true, y_predict, target_names=target_names))
print ("**************************************************************")

plt.figure()
cnf_matrix = confusion_matrix(y_true, y_predict)
plot_confusion_matrix(cnf_matrix, classes=target_names,normalize=True,
                    title='confusion matrix')

plt.show()